# Ultimate Guitar Scraping. First part: extracting filter items

Let's create a Selenium Driver to access the page

In [8]:
import requests
from selenium import webdriver
import os
import re
import codecs
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options 

HTML_PATH = '/Users/xf20939/Desktop/HALLELUJAH CHORDS (ver 2) by Jeff Buckley @ Ultimate-Guitar.Com.html'
#CHORD_CLASS_NAME = '_3PpPJ OrSDI'
URL = 'https://tabs.ultimate-guitar.com/tab/ed-sheeran/perfect-chords-1956589'



TODOs:

1) change my function to populate a dataframe given a LIST
2) instead of giving me chord list, make jack's code give me out a dataframe

In [37]:



def generate_list_of_chord_page_links(main_page):
    #takes in a link containing a list of pages, finds all links to chord pages, returns list of these links.
    CHORD_PAGE_LINK_CLASSNAME = '_3DU-x JoRLr _3dYeW'
    main_soup = generateSoupOfWebpage(main_page_url)
    links_to_chord_page = main_soup.find_all('a', { 'class' : CHORD_PAGE_LINK_CLASSNAME}, href=True)
    list_of_chord_page_links = []
    for chord_page in links_to_chord_page:
        list_of_chord_page_links.append(chord_page['href'])
    return list_of_chord_page_links



In [38]:
#main_page_url = 'https://www.ultimate-guitar.com/explore?order=hitstotal_desc&type[]=Chords'
main_page_url ='https://www.ultimate-guitar.com/explore?order=hitstotal_desc&page=2&type[]=Chords'
generate_list_of_chord_page_links(main_page_url)

['https://tabs.ultimate-guitar.com/tab/coldplay/fix-you-chords-202594',
 'https://tabs.ultimate-guitar.com/tab/adele/when-we-were-young-chords-1782038',
 'https://tabs.ultimate-guitar.com/tab/john-denver/take-me-home-country-roads-chords-1101747',
 'https://tabs.ultimate-guitar.com/tab/misc-soundtrack/a-star-is-born-shallow-chords-2480119',
 'https://tabs.ultimate-guitar.com/tab/snow-patrol/chasing-cars-chords-355425',
 'https://tabs.ultimate-guitar.com/tab/james-arthur/say-you-wont-let-go-chords-1874747',
 'https://tabs.ultimate-guitar.com/tab/bruno-mars/grenade-chords-993953',
 'https://tabs.ultimate-guitar.com/tab/the-fray/how-to-save-a-life-chords-258804',
 'https://tabs.ultimate-guitar.com/tab/frank-sinatra/fly-me-to-the-moon-chords-335196',
 'https://tabs.ultimate-guitar.com/tab/the-cranberries/zombie-chords-844902',
 'https://tabs.ultimate-guitar.com/tab/misc-cartoons/frozen-let-it-go-chords-1445224',
 'https://tabs.ultimate-guitar.com/tab/rihanna/stay-chords-1195964',
 'https:/

In [13]:
                 

def generateSoupOfWebpage(url):   
    #takes in a URL, returns its soup
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    driver = webdriver.Chrome("./chromedriver", options=options)
    driver.get(url)
    html_source_code = driver.execute_script("return document.body.innerHTML;")
    driver.close()

    return BeautifulSoup(html_source_code, 'html.parser')


def generateSoupForSetOfWebpages(url_list):
    #takes in a list of urls. returns a list of page soups.
    soups = []    
    for url in url_list:
        soups.append(generateSoupOfWebpage(url))  
    return soups

# List of pages we want to scrape chords from
url = [
    'https://tabs.ultimate-guitar.com/tab/the-beatles/let-it-be-chords-60690',
    'https://tabs.ultimate-guitar.com/tab/ed-sheeran/perfect-chords-1956589',
    'https://tabs.ultimate-guitar.com/tab/adele/easy-on-me-chords-3911342'
]

# Fetches the URLS and returns the pages as a list of BeutifulSoup objects
soups = generateSoupForSetOfWebpages(url)


In [10]:

def getSongInfo(page_soup):
    #takes in the soup of HTML and the class name for the HTML class in which the song info is kept
    INFO_CLASS_NAME = '_3rlxz'
    song_info_spans = page_soup.find_all('span', { 'class' : INFO_CLASS_NAME }) #finds span objects with the correct class
    song_info = [str(i.contents[0]) for i in song_info_spans] #finds contents of the span objects
    for stri in song_info:
        if '<span' in stri:
            song_info.remove(stri) #removes some excess spans that weirdly ended up in there... check on other songs
    #' '.join(song_info)
    return song_info

def extractChordsFromPageSoup(page_soup):
    #Given a specific path to a HTML file, convert the HTML into a list of chords. Function takes in the BeautifulSoup object for a chord page, and the class name of spans in which chords are kept.
    CHORD_CLASS_NAME = '_3PpPJ OrSDI' # this can be changed if UltimateGuitar tweak the site
    spans_containg_chords = page_soup.find_all('span', { 'class' : CHORD_CLASS_NAME })
    song_chords = [i.contents[0] for i in spans_containg_chords]
    return song_chords

def createDF(global_dataframe, title, author, song_information, chord_information):
    #takes in the global data frame, and all the information needed to populate the rows of the df. appends this information on to the global dataframe and returns the dataframe.
    local_dataframe = pd.DataFrame()
    local_dataframe['title'] = [title]
    local_dataframe['author'] = [author]
    local_dataframe['song_information'] = [' '.join(song_information)]
    local_dataframe['chords'] = [chord_information]
    return global_dataframe.append(local_dataframe)
    
    

In [11]:
chords_df = pd.DataFrame() #this is the 'global df'
#chord_page_soup = getSoup(HTML_PATH)

for soup in soups:
    title = soup.find('h1').get_text()
    chords = extractChordsFromPageSoup(soup)
    song_info = getSongInfo(soup)
    author = soup.find_all('div', { 'class' : "_-8l1v" })[0].get_text()
    chords_df = createDF(chords_df, title, author, song_info, chords)



chords_df

In [12]:
chords_df # so this is working byt appending all the same chords, so there is something wrong in the loop. also, need to append the author- is it in the h2 tags?

,title,author,song_information,chords
0,Let It Be chords,Let It Be – The Beatles,Let It Be\r\n Written and performed by: The Be...,"[C, G, Am, Am7, Fmaj7, F6, C, G, F, E, D, D, C..."
0,Perfect chords,Perfect – Ed Sheeran,Capo: 1st fret\r\n Play: G\r\n Key: Ab\r\n \r...,"[G, G, Em, C, D, G, Em, C, D, G, Em, C, G, D, ..."
0,Easy On Me chords,Easy On Me – Adele,Video here - https://youtu.be/U3ASj1L6_sY\r\n ...,"[C, Am7, Em, Fmaj7, G, C, Am7, Em, Fmaj7, F, F..."


In [518]:
chords_df.iloc[2]['song_information']

['Video here - https://youtu.be/U3ASj1L6_sY\r\n',
 ' \r\n',
 '[Intro - Piano Arranged for Guitar]\r\n',
 ' \r\n',
 'e|--------------------|--------------------|\r\n',
 'B|1---1-0-1-3-1-0---1-|1---1-0-1-3-1-0---1-|\r\n',
 'G|0---0-----0-0-0---0-|2---2-----2-2-2---2-|\r\n',
 'D|--2-----------------|2-2-----------------|\r\n',
 'A|3---------3-----3---|0---------0-----0---|\r\n',
 'E|--------------------|--------------------|\r\n',
 ' \r\n',
 'e|--------------------|--------------------|\r\n',
 'B|1--0--1--0--1--0----|1-------------------|\r\n',
 'G|0--0--0--0--0--0----|0--2----------------|\r\n',
 'D|--------------------|---3----------------|\r\n',
 'A|--------------------|--------------------|\r\n',
 'E|0-----0-----0-------|1-------------------|\r\n',
 ' \r\n',
 '[Verse 1]\r\n',
 '            ',
 "There ain't no gold in this river\r\n",
 '                ',
 "That I've been washing my hands in forever\r\n",
 'I know there is hope in these waters\r\n',
 '             ',
 "But I can't brin

In [357]:
chords_df

,title,song_information,chords
0,HALLELUJAH CHORDS (ver 2) by Jeff Buckley @ Ul...,[Capo 1 for the official video https://www.yo...,"[C, Am, C, Am, C, Am, C, Am, F, G, C, G, C, F,..."


In [342]:
print(chord_page_soup.find('title').contents[0])

PERFECT CHORDS by Ed Sheeran @ Ultimate-Guitar.Com


### Let's create functions ion order to save data

! rm data.h5

import h5py
hf = h5py.File('data.h5', 'w')

hf.('genres', data=genres)


In [25]:
import pickle
 
with open('genres', 'wb') as genres_file:
  pickle.dump(genres, genres_file)

In [26]:
genres_loaded = 5
with open('genres', 'rb') as genres_file:
    genres_loaded = pickle.load(genres_file)

print_list(genres_loaded)

['FilterItem:code=4, name=Rock, filter_pattern=&&genres[]=4',
 'FilterItem:code=666, name=Folk, filter_pattern=&&genres[]=666',
 'FilterItem:code=14, name=Pop, filter_pattern=&&genres[]=14',
 'FilterItem:code=49, name=Country, filter_pattern=&&genres[]=49',
 'FilterItem:code=16, name=Electronic, filter_pattern=&&genres[]=16',
 'FilterItem:code=70, name=Rhythm And Blues, filter_pattern=&&genres[]=70',
 'FilterItem:code=8, name=Metal, filter_pattern=&&genres[]=8',
 'FilterItem:code=434, name=Contemporary R&b, filter_pattern=&&genres[]=434',
 'FilterItem:code=1016, name=Religious Music, filter_pattern=&&genres[]=1016',
 'FilterItem:code=45, name=Hip Hop, filter_pattern=&&genres[]=45',
 'FilterItem:code=19, name=Reggae, filter_pattern=&&genres[]=19',
 'FilterItem:code=84, name=Jazz, filter_pattern=&&genres[]=84',
 'FilterItem:code=99, name=Blues, filter_pattern=&&genres[]=99',
 'FilterItem:code=195, name=World Music, filter_pattern=&&genres[]=195',
 'FilterItem:code=85, name=Disco, filter_

In [27]:
%%writefile jl_io.py

import pickle

def to_file(obj, path):
    with open(path, 'wb') as file:
          pickle.dump(obj, file) 
            
def from_file(path):
    with open(path, 'rb') as file:
        return pickle.load(file)    

Overwriting jl_io.py


Written to a new file to be reused in later notebooks

In [28]:
import jl_io as io

In [31]:
io.to_file(genres, 'prueba')
print_list(io.from_file('prueba'))

['FilterItem:code=4, name=Rock, filter_pattern=&&genres[]=4',
 'FilterItem:code=666, name=Folk, filter_pattern=&&genres[]=666',
 'FilterItem:code=14, name=Pop, filter_pattern=&&genres[]=14',
 'FilterItem:code=49, name=Country, filter_pattern=&&genres[]=49',
 'FilterItem:code=16, name=Electronic, filter_pattern=&&genres[]=16',
 'FilterItem:code=70, name=Rhythm And Blues, filter_pattern=&&genres[]=70',
 'FilterItem:code=8, name=Metal, filter_pattern=&&genres[]=8',
 'FilterItem:code=434, name=Contemporary R&b, filter_pattern=&&genres[]=434',
 'FilterItem:code=1016, name=Religious Music, filter_pattern=&&genres[]=1016',
 'FilterItem:code=45, name=Hip Hop, filter_pattern=&&genres[]=45',
 'FilterItem:code=19, name=Reggae, filter_pattern=&&genres[]=19',
 'FilterItem:code=84, name=Jazz, filter_pattern=&&genres[]=84',
 'FilterItem:code=99, name=Blues, filter_pattern=&&genres[]=99',
 'FilterItem:code=195, name=World Music, filter_pattern=&&genres[]=195',
 'FilterItem:code=85, name=Disco, filter_

### We finally save the data

In [32]:
! ls

001_Exploring_data.ipynb chromedriver             jl_io.py
Scraping.ipynb           data.h5                  prueba
Scraping_02.ipynb        decades                  styles
__pycache__              genres


In [33]:
! rm genres prueba

In [34]:
! ls

001_Exploring_data.ipynb __pycache__              decades
Scraping.ipynb           chromedriver             jl_io.py
Scraping_02.ipynb        data.h5                  styles


In [43]:
! mkdir ../data

mkdir: ../data: File exists


In [54]:
io.to_file([genre.to_dictionary() for genre in genres],'../data/genres')
io.to_file([style.to_dictionary() for style in styles],'../data/styles')
io.to_file([decade.to_dictionary() for decade in decades],'../data/decades')

In [55]:
!ls 

001_Exploring_data.ipynb __pycache__              jl_io.py
Scraping.ipynb           chromedriver
Scraping_02.ipynb        data.h5


In [56]:
print_list(io.from_file('../data/genres'))

["{'code': '4', 'name': 'Rock', 'pattern': '&&genres[]=4'}",
 "{'code': '666', 'name': 'Folk', 'pattern': '&&genres[]=666'}",
 "{'code': '14', 'name': 'Pop', 'pattern': '&&genres[]=14'}",
 "{'code': '49', 'name': 'Country', 'pattern': '&&genres[]=49'}",
 "{'code': '16', 'name': 'Electronic', 'pattern': '&&genres[]=16'}",
 "{'code': '70', 'name': 'Rhythm And Blues', 'pattern': '&&genres[]=70'}",
 "{'code': '8', 'name': 'Metal', 'pattern': '&&genres[]=8'}",
 "{'code': '434', 'name': 'Contemporary R&b', 'pattern': '&&genres[]=434'}",
 "{'code': '1016', 'name': 'Religious Music', 'pattern': '&&genres[]=1016'}",
 "{'code': '45', 'name': 'Hip Hop', 'pattern': '&&genres[]=45'}",
 "{'code': '19', 'name': 'Reggae', 'pattern': '&&genres[]=19'}",
 "{'code': '84', 'name': 'Jazz', 'pattern': '&&genres[]=84'}",
 "{'code': '99', 'name': 'Blues', 'pattern': '&&genres[]=99'}",
 "{'code': '195', 'name': 'World Music', 'pattern': '&&genres[]=195'}",
 "{'code': '85', 'name': 'Disco', 'pattern': '&&genres[

In [57]:
print_list(io.from_file('../data/styles'))

["{'code': '24', 'name': 'Pop Rock', 'pattern': '&&subgenres[]=24'}",
 "{'code': '665', 'name': 'Singer-songwriter', 'pattern': '&&subgenres[]=665'}",
 "{'code': '3', 'name': 'Alternative Rock', 'pattern': '&&subgenres[]=3'}",
 "{'code': '9', 'name': 'Indie Rock', 'pattern': '&&subgenres[]=9'}",
 "{'code': '46', 'name': 'Indie Pop', 'pattern': '&&subgenres[]=46'}",
 "{'code': '1', 'name': 'Pop Punk', 'pattern': '&&subgenres[]=1'}",
 "{'code': '7', 'name': 'Hard Rock', 'pattern': '&&subgenres[]=7'}",
 "{'code': '813', 'name': 'Contemporary Folk', 'pattern': '&&subgenres[]=813'}",
 "{'code': '94', 'name': 'Folk Rock', 'pattern': '&&subgenres[]=94'}",
 "{'code': '1093', 'name': 'Folk Pop', 'pattern': '&&subgenres[]=1093'}",
 "{'code': '1087', 'name': 'Contemporary Country', 'pattern': '&&subgenres[]=1087'}",
 "{'code': '197', 'name': 'Country Pop', 'pattern': '&&subgenres[]=197'}",
 "{'code': '104', 'name': 'Electropop', 'pattern': '&&subgenres[]=104'}",
 "{'code': '82', 'name': 'Indie Fo

In [58]:
print_list(io.from_file('../data/decades'))

["{'code': '2010', 'name': '2010s', 'pattern': '&&decade[]=2010'}",
 "{'code': '2000', 'name': '2000s', 'pattern': '&&decade[]=2000'}",
 "{'code': '1990', 'name': '1990s', 'pattern': '&&decade[]=1990'}",
 "{'code': '1980', 'name': '1980s', 'pattern': '&&decade[]=1980'}",
 "{'code': '1970', 'name': '1970s', 'pattern': '&&decade[]=1970'}",
 "{'code': '1960', 'name': '1960s', 'pattern': '&&decade[]=1960'}",
 "{'code': '1950', 'name': '1950s', 'pattern': '&&decade[]=1950'}"]